# Creating containers and shipments via the Transcriptic REST API

In [2]:
import requests
import json
import autoprotocol as ap

## 1. Set your Auth headers and endpoint

In [24]:
headers = {"X-User-Email": 'dbryce@sift.net',  # user-account-email
           "X-User-Token": 'F8bPPxtAxLxgknrRnrya',  # Regular-mode API key
           "Content-Type": "application/json",
           "Accept": "application/json"}

organization = 'sd2org' #put your org name here, you can find it in the URL when logged into the web app.
url = "https://secure.transcriptic.com/sd2org/inventory/samples/create_with_shipment"

## 2. Build your shipment object

Create a dict containing an array of `containers`. Each container will have a dict of `aliquots` zero-indexed. Aliquots require a `name` and a `volume_ul`. The properties dict can be empty but it can also be populated with key value pairs to further describe the well. Features like compound ID, concentration and solvent can be useful.


In [18]:
example_shipment_object = json.loads("""{
  "containers": [{
    "label": "danb_test_plate",
    "container_type": "384-flat-white-white-proxiplate-plus",
    "storage_condition": "cold_4",
    "test_mode": true,
    "aliquots": {
      "0": {
        "name": "Chlorodibromethane",
        "volume_ul": 8,
        "properties": {
        }
      },
      "1": {
        "name": "Dimefuron",
        "volume_ul": 8,
        "properties": {
        }
      },
      "2": {
        "name": "o-Chloronitrobenzene",
        "volume_ul": 8,
        "properties": {
        }
      }
    }
  }]
}""")

body = json.dumps(example_shipment_object)
body

'{"containers": [{"label": "danb_test_plate", "container_type": "384-flat-white-white-proxiplate-plus", "storage_condition": "cold_4", "test_mode": true, "aliquots": {"0": {"name": "Chlorodibromethane", "volume_ul": 8, "properties": {}}, "1": {"name": "Dimefuron", "volume_ul": 8, "properties": {}}, "2": {"name": "o-Chloronitrobenzene", "volume_ul": 8, "properties": {}}}}]}'

# POST the request

HTTP POST the request to the `create_with_shipment` endpoint and collect the response which sould return `201 Created`.

In [25]:
# POST body to the run endpoint
print(url)
print(body)
print(headers)
response = requests.post(url, body, headers=headers)
response

https://secure.transcriptic.com/sd2org/inventory/samples/create_with_shipment
{"containers": [{"label": "danb_test_plate", "container_type": "384-flat-white-white-proxiplate-plus", "storage_condition": "cold_4", "test_mode": true, "aliquots": {"0": {"name": "Chlorodibromethane", "volume_ul": 8, "properties": {}}, "1": {"name": "Dimefuron", "volume_ul": 8, "properties": {}}, "2": {"name": "o-Chloronitrobenzene", "volume_ul": 8, "properties": {}}}}]}
{'X-User-Email': 'dbryce@sift.net', 'X-User-Token': 'F8bPPxtAxLxgknrRnrya', 'Content-Type': 'application/json', 'Accept': 'application/json'}


<Response [201]>

Parse the response contents

In [26]:
resp = json.loads(response.content)

In [27]:
resp

{'containers': [{'id': 'ct1eb96btb4gjuq',
   'container_type_id': '384-flat-white-white-proxiplate-plus',
   'barcode': None,
   'deleted_at': None,
   'created_at': '2020-04-13T18:14:16.440-07:00',
   'accessions': 0,
   'organization_id': 'org1amxh23ednpz',
   'slot': None,
   'cover': None,
   'test_mode': True,
   'label': 'danb_test_plate',
   'location_id': None,
   'shipment_id': 'sr1eb96btbk95j5',
   'storage_condition': 'cold_4',
   'shipment_code': 'HNX',
   'status': 'inbound',
   'expires_at': None,
   'container_type': {'id': '384-flat-white-white-proxiplate-plus',
    'name': '384-well flat-bottom polystyrene proximity plus plate',
    'well_count': 384,
    'well_depth_mm': '5.3',
    'well_volume_ul': '28.0',
    'capabilities': ['incubate',
     'seal',
     'unseal',
     'image_plate',
     'stamp',
     'dispense',
     'spin',
     'flash_freeze',
     'echo_dest',
     'liquid_handle',
     'cover',
     'fluorescence',
     'luminescence',
     'pipette',
     'u

Access the shipment

In [85]:
resp["shipment"]

{'id': 'sr1chmnspg3vrm',
 'shipment_type': 'sample',
 'organization_id': 'org18qrjdh7jn27',
 'data': {},
 'checked_in_at': None,
 'created_at': '2019-03-02T11:11:16.595-08:00',
 'label': 'TWZU',
 'editable': True,
 'name': None,
 'packing_url': None,
 'created_by': None,
 'user_id': 'u16r2fqgpp93m',
 'status': 'pending',
 'organization': {'id': 'org18qrjdh7jn27', 'name': 'Swift on Pharma'},
 'user': {'id': 'u16r2fqgpp93m',
  'first_name': 'Ben',
  'last_name': 'Miles',
  'profile_img_url': 'https://static-public.transcriptic.com/profile_pics/6197c3d7-ab0a-41ac-b3b2-59c173d00e02/profile_pic.jpg',
  'name': 'Ben Miles'}}

Access the array of containers in the shipment

In [87]:
containers = resp["containers"]

You will want to pull out the 3 letter identifier to be physically added to the corresponding container.

In [88]:
for container in containers:
    print(container["label"], "->", container["shipment_code"])

my_plate -> TAR


Then grab the accessioning code which will go on the shipping label

In [89]:
print(f'''Transcriptic, Inc
      \nATTN: Accessioning {resp['shipment']["label"]}
      \n3565 Haven Avenue Suite 3
      \nMenlo Park, CA 94025''')

Transcriptic, Inc
      
ATTN: Accessioning TWZU
      
3565 Haven Avenue Suite 3
      
Menlo Park, CA 94025


In [90]:
p = ap.Protocol()
ben = p.ref("my_plate",
            cont_type="96-flat",
            storage="cold_4",
            discard=False)
print(ben.name)
for well in ben.all_wells():
    well.name = "cool_compound"
    well.volume = "25:uL"
float(ap.Unit(ben.all_wells()[0].volume).magnitude)

my_plate


25.0

In [91]:
ben.container_type

ContainerType(name='96-well flat-bottom plate', is_tube=False, well_count=96, well_depth_mm=None, well_volume_ul=Unit(340.0, 'microliter'), well_coating=None, sterile=False, cover_types=['low_evaporation', 'standard', 'universal'], seal_types=None, capabilities=['liquid_handle', 'spin', 'absorbance', 'fluorescence', 'luminescence', 'incubate', 'gel_separate', 'gel_purify', 'cover', 'dispense'], shortname='96-flat', col_count=12, dead_volume_ul=Unit(25, 'microliter'), safe_min_volume_ul=Unit(65, 'microliter'), true_max_vol_ul=Unit(340.0, 'microliter'), vendor='Corning', cat_no='3632', prioritize_seal_or_cover='seal')

In [75]:
def create_shipment(refs):
    containers = []
    for ref in refs:
        aliquots = []
        for well in ref.all_wells():
            aliquot = {
                str(well.index): {
                    "name": well.name,
                    "volume_ul": float(ap.Unit(well.volume).magnitude),
                    "properties": {}
                }
            }
            aliquots.append(aliquot)
        container = {
            "label": ref.name,
            "container_type": ref.container_type,
            "storage_condition": ref.storage,
            "test_mode": False,
            "aliquots": aliquots
        }
        containers.append(container)
    shipment_object = containers
    return shipment_object

In [76]:
ship = create_shipment([ben])

In [71]:
ship[0]

{'label': 'my_plate',
 'container_type': '384-flat-white-white-proxiplate-plus',
 'storage_condition': 'cold_4',
 'test_mode': False,
 'aliquots': [{'0': {'name': 'alex', 'volume_ul': 25.0, 'properties': {}}},
  {'1': {'name': 'alex', 'volume_ul': 25.0, 'properties': {}}},
  {'2': {'name': 'alex', 'volume_ul': 25.0, 'properties': {}}},
  {'3': {'name': 'alex', 'volume_ul': 25.0, 'properties': {}}},
  {'4': {'name': 'alex', 'volume_ul': 25.0, 'properties': {}}},
  {'5': {'name': 'alex', 'volume_ul': 25.0, 'properties': {}}},
  {'6': {'name': 'alex', 'volume_ul': 25.0, 'properties': {}}},
  {'7': {'name': 'alex', 'volume_ul': 25.0, 'properties': {}}},
  {'8': {'name': 'alex', 'volume_ul': 25.0, 'properties': {}}},
  {'9': {'name': 'alex', 'volume_ul': 25.0, 'properties': {}}},
  {'10': {'name': 'alex', 'volume_ul': 25.0, 'properties': {}}},
  {'11': {'name': 'alex', 'volume_ul': 25.0, 'properties': {}}},
  {'12': {'name': 'alex', 'volume_ul': 25.0, 'properties': {}}},
  {'13': {'name': '

In [4]:
import transcriptic as tx

In [15]:
from transcriptic.config import Connection
api = Connection.from_file("~/.transcriptic")


In [17]:
my_cont = tx.container("ct1ce7rfxvata2")

/anaconda3/envs/my-rdkit-env/lib/python3.6/site-packages/transcriptic-5.6.5-py3.6.egg/transcriptic/jupyter/objects.py:886: UserWarning: ContainerType given is not supported yet in AP-Py
  warnings.warn("ContainerType given is not supported yet in AP-Py")


In [18]:
my_cont

Container(Plate 1_test_ben)

In [20]:
my_cont.attributes["properties"]

{}

In [21]:
my_cont.attributes["properties"] = {"tare_mass": "1.0:gram"}

In [22]:
my_cont.attributes["properties"]

{'tare_mass': '1.0:gram'}